In [ ]:
# Only run this once to install the necessary packages
%pip install torch
%pip install pandas
%pip install sklearn
%pip install scipy

In [ ]:
import torch
import pandas as pd
from sklearn.preprocessing import normalize
from IPython.display import clear_output
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
data = pd.read_csv("books_data.csv", nrows=40000)
data['Title'] = data['Title'].fillna('Unknown')
data['categories'] = data['categories'].fillna('Unknown')
data['description'] = data['description'].fillna('')
data['description'] = data['description'].apply(lambda x: x.lower())
data['book_content'] = (
    (data['Title'] + ' ') * 2
    + data['description'] + ' '
    + data['authors'].apply(lambda x: ' '.join(x) if isinstance(x, list) else '') + ' '
    + data['categories'].apply(lambda x: ' '.join(x) * 5 if isinstance(x, list) else '')
)
data['book_content'] = data['book_content'].str.replace(r'[^\w\s]', '', regex=True).str.lower()

In [ ]:
book_embeddings = torch.load('bert_embeddings.pt')
print(book_embeddings.shape)

In [ ]:
normalized_book_embeddings = normalize(book_embeddings)

In [ ]:
cosine_sim_matrix = cosine_similarity(normalized_book_embeddings)

In [ ]:
def recommend_books_by_cosine(book_title, threshold, cosine_sim_matrix):
    idx = data[data['Title'] == book_title].index[0]
    
    # Compute the cosine similarity scores
    sim_scores = list(enumerate(cosine_sim_matrix[idx]))
    
    # Sort the books based on cosine similarity (higher similarity means more similar)
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = [(i, "{:.5f}".format(score)) for i, score in sim_scores if score >= threshold]

    # Get the book titles and their similarity scores
    book_recommendations = [(data['Title'].iloc[i[0]], i[1]) for i in sim_scores]

    return book_recommendations

In [ ]:
book_title = ''
while (book_title != 'q'):
  clear_output(wait=True)
  book_title = input("Enter the title of a book: ")
  recommended_books = recommend_books_by_cosine(book_title, 0.4, cosine_sim_matrix)
  f = open('cosine_output.txt', 'w')
  f.write('Counts: ' + str(len(recommended_books)) + '\n\n')
  for book in recommended_books:
    f.write(book[1] + ' | ' + str(book[0]) + '\n')

  print('Found: ' + str(len(recommended_books)))